# CARLINI WAGNER #

Ricordati di cambiare le cartelle del test set.

#### *Import* 

In [4]:
import os
import re
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from torchvision import transforms
import torch
from torch.nn import CrossEntropyLoss
import torch.nn as nn
from art.estimators.classification import PyTorchClassifier
import tensorflow as tf
from matplotlib import pyplot as plt
# Import all L-distance based attacks
from art.attacks.evasion import CarliniL2Method, CarliniL0Method, CarliniLInfMethod
from SENet import SENet
from utils import get_labels
from utils import load_state_dict
from utils import load_image_NN1
from utils import load_test_image_NN2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = str(device)

c:\Users\cassd\miniconda3\envs\ai4cyber\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
if device in 'cuda':
    print("Import Inception.Inception")
    import inception
    resnet = inception.InceptionResnetV1(pretrained='vggface2').eval()
elif device == "cpu":
    print("Import Facenet.Inception")
    from facenet_pytorch import InceptionResnetV1
    resnet = InceptionResnetV1(pretrained='vggface2').eval()

Import Inception.Inception


#### Inizializzazione

In [6]:
# ----------------------------Resnet-----------------------
resnet.classify = True
resnet.to(device)
fpath = tf.keras.utils.get_file('rcmalli_vggface_labels_v2.npy',
                             "https://github.com/rcmalli/keras-vggface/releases/download/v2.0/rcmalli_vggface_labels_v2.npy",
                             cache_subdir="./")
model_resnet = PyTorchClassifier(resnet,input_shape=[224,224], loss=CrossEntropyLoss(),nb_classes=8631, device_type=device) #This class implements a classifier with the PyTorch framework.

# ---------------------Senet-----------------------------
senet = SENet.senet50(num_classes=8631, include_top=True)
load_state_dict(senet,'senet50_scratch_weight.pkl')
senet.eval()
senet.to(device)
model_senet = PyTorchClassifier(senet,input_shape=[224,224], loss=CrossEntropyLoss(),nb_classes=8631, device_type=device)


#--------------------------------------------------------
LABELS = get_labels()

def load_image(filename):
    img = Image.open(filename)
    rsz = img.resize((160, 160))
    tns = transforms.ToTensor()(rsz)
    tns.to(device)
    return tns


#### SIGNIFICATO PARAMETRI 


Il metodo Carlini and Wagner L_0 Attack è un attacco iterativo che mira a trovare un esempio avversario minimizzando il numero di caratteristiche modificate (norma L_0). Ecco una spiegazione dettagliata del metodo e dei suoi parametri:

Metodo Carlini and Wagner L_0 Attack
Parametri del Metodo __init__

classifier (CLASSIFIER_CLASS_LOSS_GRADIENTS_TYPE):
Un classificatore addestrato che implementa i metodi per calcolare le perdite e i gradienti delle perdite rispetto agli input.

confidence (float = 0.0):
La fiducia degli esempi avversari: un valore più alto produce esempi che sono più lontani dall'input originale ma classificati con maggiore fiducia come la classe target. Questo parametro bilancia la necessità di modificare l'input originale con la certezza che l'output sia nella classe avversaria desiderata.

targeted (bool = False):
Indica se l'attacco è mirato a una specifica classe (True) o meno (False). In un attacco mirato, l'obiettivo è far classificare l'input avversario come una classe specifica scelta dall'attaccante. In un attacco non mirato, l'obiettivo è semplicemente far classificare l'input in una classe diversa da quella corretta.

learning_rate (float = 0.01):
Il tasso di apprendimento iniziale per l'algoritmo di attacco. Valori più piccoli producono risultati migliori ma convergono più lentamente.

binary_search_steps (int = 10):
Numero di volte in cui regolare la costante con la ricerca binaria (valore positivo). Se binary_search_steps è grande, l'algoritmo non è molto sensibile al valore di initial_const. Questo parametro controlla quante volte la costante di trade-off viene aggiustata per trovare il miglior equilibrio tra la distanza e la fiducia.

max_iter (int = 10):
Il numero massimo di iterazioni per l'attacco.

initial_const (float = 0.01):
La costante di trade-off iniziale c per regolare l'importanza relativa tra la distanza e la fiducia. Se binary_search_steps è grande, il valore iniziale di questa costante non è critico.

mask (ndarray | None = None):
Le caratteristiche iniziali che possono essere modificate dall'algoritmo. Se non specificato, l'algoritmo utilizza l'intero set di caratteristiche. Questo parametro può essere utilizzato per limitare l'attacco a modificare solo determinate parti dell'input.

warm_start (bool = True):
Invece di iniziare la discesa del gradiente in ogni iterazione dall'immagine iniziale, si inizia la discesa del gradiente dalla soluzione trovata nell'iterazione precedente. Questo parametro può accelerare la convergenza.

max_halving (int = 5):
Numero massimo di passi di dimezzamento nella ricerca lineare di ottimizzazione. Questo parametro controlla quante volte la dimensione del passo viene dimezzata durante la ricerca di un passo ottimale.

max_doubling (int = 5):
Numero massimo di passi di raddoppiamento nella ricerca lineare di ottimizzazione. Questo parametro controlla quante volte la dimensione del passo viene raddoppiata durante la ricerca di un passo ottimale.
batch_size (int = 1):

La dimensione del batch su cui vengono generati i campioni avversari. Questo parametro determina quanti campioni vengono processati contemporaneamente durante l'attacco.

verbose (bool = True):
Mostra barre di avanzamento. Se impostato su True, verranno visualizzati i progressi dell'attacco durante l'esecuzione.

### *Attacco* **NON TARGETED**

In [27]:
dataset_dir = "test_set_cropped_piccolo/" 
binary_search_steps = 2
confidence = 0.8
max_iter = [1]
learning_rate = [0.1,0.2,0.5,0.7,0.9]
initial_const = [1,300]


correct_predictions_resnet = 0
accuracy_for_learning_rate_resnet = np.zeros((len(initial_const),len(learning_rate)))        #riga
perturbation_for_learning_rate_resnet = np.zeros((len(initial_const),len(learning_rate)))

correct_predictions_senet = 0
accuracy_for_learning_rate_senet = np.zeros((len(initial_const),len(learning_rate)))        #riga
#perturbation_for_learning_rate_senet = np.zeros((len(initial_const),len(learning_rate)))

total_images = 0
print("Inizio Attacco CARLINI-WAGNER NON-TARGETED")

for i in range(len(initial_const)):
        for j in range(len(learning_rate)):   #Se qualcosa funziona strano controllare questo zip
            correct_predictions_resnet = 0
            correct_predictions_senet = 0
            total_images = 0
            perturbation = []
            
            attack = CarliniL2Method(classifier=model_resnet, binary_search_steps=binary_search_steps, confidence=confidence, max_iter=max_iter[0], learning_rate=learning_rate[j], initial_const=initial_const[i], targeted=False)
            
            print("<---> Attacco con learning rate = {} e initial cost = {} <--->".format(learning_rate[j],initial_const[i]))
            for filename in os.listdir(dataset_dir):
                if filename.endswith(".jpg") or filename.endswith(".jpeg"):
                    person_path = os.path.join(dataset_dir, filename)
                    test_img = load_image_NN1(person_path,device)
                    test_img = test_img.numpy()
                    
                    test_images_adv = attack.generate(test_img)
                    resnet_predictions = model_resnet.predict(test_images_adv)

                    test_images_NN2 = load_test_image_NN2(test_images_adv)
                    senet_predictions = model_senet.predict(test_images_NN2)

                    correct_label = re.sub(r'_\d+_face_0\.jpg$', '', filename)
                    #print("Etichetta corretta:", correct_label)   
                    perturbation.append(np.mean(np.abs((test_images_adv - test_img))))  #Salvo le perturbazioni applicate su ogni immagine
            
                    predicted_label_resnet = LABELS[np.array(resnet_predictions.argmax())]
                    predicted_label_senet = LABELS[np.array(senet_predictions.argmax())]
                    #print("Predetto {} con probabilità {} e con perturbazione {}".format(predicted_label,model_predictions[0][model_predictions.argmax()],perturbation[-1]))
                    total_images+=1
                    
                    predicted_label_resnet = str(predicted_label_resnet)  # da togliere ?
                    predicted_label_senet = str(predicted_label_senet)
                    
                    if correct_label in predicted_label_resnet:
                        correct_predictions_resnet+=1

                    #print("Adversarial Sample misclassificati correttamente attuale: {}%".format((100-(accuracy*100))))
                    accuracy_resnet = correct_predictions_resnet/total_images

                    if correct_label in predicted_label_senet:
                        correct_predictions_senet+=1

                    accuracy_senet = correct_predictions_senet/total_images


                    if total_images == 500:
                        print("Sei a 500 Immagini initial_cost:{} | learning_rate:{}".format(initial_const[i], learning_rate[i]))

            if total_images != 0:    
                if len(perturbation) == total_images:
                    perturbazione_media = sum(perturbation)/total_images   # In media quanta pertubazione è stata applicata su ogni immagine 
                    perturbation_for_learning_rate_resnet[i][j] = perturbazione_media 
                    #print("----------- Perturbazione media aggiunta a tutte le immagini ---> {}% ----------------".format(learning,perturbazione_media))
                    
                      
                final_accuracy_resnet = correct_predictions_resnet/total_images          # Accuracy su tutte le immagini
                accuracy_for_learning_rate_resnet[i][j] = final_accuracy_resnet
                #print("----------- Accuracy sugli adversarial Sample equivale a ---> {}% ----------------".format(learning,final_accuracy))
                final_accuracy_senet = correct_predictions_senet/total_images
                accuracy_for_learning_rate_senet[i][j] = final_accuracy_senet
            else:
                print("ERRORE TOTAL IMAGE == 0 ERRORE")
                    
        learning_contatore = 0
        print("--------------------------------------------------------------------------------------------------------------------------------------------------------------------lllllllllllllllllll")
        print("")

Inizio Attacco CARLINI-WAGNER NON-TARGETED
<---> Attacco con learning rate = 0.1 e initial cost = 1 <--->


C&W L_2: 100%|██████████| 1/1 [00:00<00:00,  1.10it/s]


<---> Attacco con learning rate = 0.2 e initial cost = 1 <--->


C&W L_2: 100%|██████████| 1/1 [00:00<00:00,  1.10it/s]


<---> Attacco con learning rate = 0.5 e initial cost = 1 <--->


...

<---> Attacco con learning rate = 0.7 e initial cost = 300 <--->


C&W L_2: 100%|██████████| 1/1 [00:01<00:00,  1.01s/it]


<---> Attacco con learning rate = 0.9 e initial cost = 300 <--->


C&W L_2: 100%|██████████| 1/1 [00:01<00:00,  1.01s/it]

--------------------------------------------------------------------------------------------------------------------------------------------------------------------lllllllllllllllllll



In [29]:
#matrice righe = iterazione, colonne = learning rate
print(accuracy_for_learning_rate_resnet[1])
print(accuracy_for_learning_rate_senet[1])
print(perturbation_for_learning_rate_resnet[0])

[0.  0.  0.2 0.2 0.3]
[0.  0.  0.1 0.1 0.2]
[0.00116675 0.0013991  0.00194145 0.00163508 0.00202984]


In [ ]:

# plot accuracy/Attack strength

fig, ax = plt.subplots()
ax.plot(learning_rate, perturbation_for_learning_rate[0], 'b-', label='Perturbazione')
ax.plot(learning_rate, accuracy_for_learning_rate[0], 'r-', label='Accuracy')
#ax.plot(np.array(learning_rate), np.array(perturbation_for_learning_rate[1]), 'r-', label='max_iter == 2')
#ax.plot(np.array(learning_rate), np.array(perturbation_for_learning_rate[2]), 'g-.', label='max_iter == 4')

legend = ax.legend(loc='upper center', shadow=True, fontsize='large')
legend.get_frame().set_facecolor('#00FFCC')

plt.ylabel('Perturbation')
plt.xlabel('Learning Rate')



plt.tight_layout()

plt.show()

#### Grafico Accuracy media a ogni iterazione

In [ ]:


# plot accuracy/Attack strength

fig, ax = plt.subplots()
ax.plot(learning_rate, np.array(accuracy_for_learning_rate[0]), 'b--', label='initial cost == 1')
ax.plot(learning_rate, np.array(accuracy_for_learning_rate[1]), 'r-', label='initial cost == 300')
ax.plot(learning_rate, np.array(accuracy_for_learning_rate[2]), 'g-', label='initial cost == 700')

legend = ax.legend(loc='upper center', shadow=True, fontsize='large')
legend.get_frame().set_facecolor('#00FFCC')

plt.ylabel('Accuracy')
plt.xlabel('Learning Rate')


plt.show()

#### Grafico Accuracy media/Perturbazione media



In [ ]:
fig, ax = plt.subplots()
ax.plot(np.array(accuracy_for_max_iter), np.array(perturbation_for_max_iter), 'b--', label='NN1')

legend = ax.legend(loc='upper center', shadow=True, fontsize='large')
legend.get_frame().set_facecolor('#00FFCC')

plt.xlabel('Attack strength (eps)')
plt.ylabel('Accuracy')
plt.show()

### *Attacco* **TARGETED**

In [15]:
dataset_dir = "test_set_cropped_piccolo/" 
binary_search_steps = 3
confidence = 0.7
max_iter = [2]
learning_rate = [0.3,0.5,1]
initial_const = [300,700]

correct_predictions_resnet = 0
correct_misclassified_resnet = 0
accuracy_for_learning_rate_targeted_resnet = np.zeros((len(max_iter),len(initial_const),len(learning_rate)))        #riga-colonna
perturbation_for_learning_rate_targeted_resnet = np.zeros((len(max_iter),len(initial_const),len(learning_rate)))
accuracy_misclassified_for_learning_rate_targeted_resnet = np.zeros((len(max_iter),len(initial_const),len(learning_rate)))

correct_predictions_senet = 0
correct_misclassified_senet = 0
accuracy_for_learning_rate_targeted_senet = np.zeros((len(max_iter),len(initial_const),len(learning_rate)))        #riga
accuracy_misclassified_for_learning_rate_targeted_senet = np.zeros((len(max_iter),len(initial_const),len(learning_rate)))
#perturbation_for_learning_rate_senet = np.zeros((len(initial_const),len(learning_rate)))

target_class = 10
etichetta_target = LABELS[target_class]
print("ETICHETTA TARGET: ", etichetta_target)

targeted_labels = target_class*np.ones(LABELS.size)
one_hot_targeted_labels = tf.keras.utils.to_categorical(targeted_labels, num_classes = 8631)


print("Inizio Attacco CARLINI-WAGNER TARGETED")
for z in range(len(max_iter)):
    for i in range(len(initial_const)):
            for j in range(len(learning_rate)):   #Se qualcosa funziona strano controllare questo zip
                correct_predictions_resnet = 0
                correct_predictions_senet = 0
                correct_misclassified_resnet = 0
                correct_misclassified_senet = 0
                total_images = 0
                perturbation = []
                
                attack = CarliniL2Method(classifier=model_resnet,
                                          binary_search_steps=binary_search_steps,
                                            confidence=confidence, max_iter=max_iter[z],
                                              learning_rate=learning_rate[j],
                                                initial_const=initial_const[i],
                                                  targeted=True)
                
                print("<---> Attacco con max_iter = {}| learning rate = {} | initial cost = {} <--->".format(max_iter[z],learning_rate[j],initial_const[i]))
                for filename in os.listdir(dataset_dir):
                    person_path = os.path.join(dataset_dir, filename)
                    test_img = load_image_NN1(person_path,device)
                    test_img = test_img.numpy()
                    test_images_adv = attack.generate(test_img,one_hot_targeted_labels)
                    resnet_predictions = model_resnet.predict(test_images_adv)
                    perturbation.append(np.mean(np.abs((test_images_adv - test_img))))
                    
                    test_images_NN2 = load_test_image_NN2(test_images_adv)
                    senet_predictions = model_senet.predict(test_images_NN2)

                    correct_label = re.sub(r'_\d+_face_0\.jpg$', '', filename)  
                      #Salvo le perturbazioni applicate su ogni immagine
            
                    predicted_label_resnet = LABELS[np.array(resnet_predictions.argmax())]
                    predicted_label_senet = LABELS[np.array(senet_predictions.argmax())]
                    print("Etichetta Corretta: {} | Predetto: {}  probabilità {} perturbazione applicata {}".format(correct_label,predicted_label_resnet,resnet_predictions[0][resnet_predictions.argmax()],perturbation[-1]))
                    total_images+=1
                    
                    predicted_label_resnet = str(predicted_label_resnet)  # da togliere ?
                    predicted_label_senet = str(predicted_label_senet)
                    
                    if correct_label in predicted_label_resnet:
                        correct_predictions_resnet+=1

                    #print("Adversarial Sample misclassificati correttamente attuale: {}%".format((100-(accuracy*100))))
                    accuracy_resnet = correct_predictions_resnet/total_images
                    
                    if correct_label in predicted_label_senet:
                        correct_predictions_senet+=1

                    accuracy_senet = correct_predictions_senet/total_images
                            
                    if etichetta_target in predicted_label_resnet:  
                        correct_misclassified_resnet += 1   #Se il modello predice l'etichetta target allora è correttamente misclassificato

                    accuracy_misclassified_resnet = correct_misclassified_resnet/total_images

                    if etichetta_target in predicted_label_senet:   #se Attacco su NN2 Correttamente riuscito
                        correct_misclassified_senet += 1   

                    accuracy_misclassified_senet = correct_misclassified_senet/total_images
                    
                    #print("Adversarial Sample misclassificati correttamente attualmente: {}%".format((accuracy_misclassified)))
                    #print("Accuracy attuale: {}%".format((correct_predictions/total_images)*100))
                
                    if total_images == 500:
                        print("Sei a 500 Immagini initial_cost:{} | learning_rate:{}".format(initial_const[i], learning_rate[i]))

            if total_images != 0:    
                if len(perturbation) == total_images:
                      # In media quanta pertubazione è stata applicata su ogni immagine
                    perturbation_for_learning_rate_targeted_resnet[z][i][j] = (sum(perturbation)/total_images)
                    
                
   
                      
                final_accuracy_resnet = correct_predictions_resnet/total_images          # Accuracy su tutte le immagini
                accuracy_for_learning_rate_targeted_senet[z][i][j] = final_accuracy_resnet

                accuracy_misclassified_resnet = correct_misclassified_resnet/total_images
                accuracy_misclassified_for_learning_rate_targeted_resnet[z][i][j] = accuracy_misclassified_resnet

                #print("----------- Accuracy sugli adversarial Sample equivale a ---> {}% ----------------".format(learning,final_accuracy))
                final_accuracy_senet = correct_predictions_senet/total_images
                accuracy_for_learning_rate_senet[z][i][j] = final_accuracy_senet

                accuracy_misclassified_senet = correct_misclassified_senet/total_images
                accuracy_misclassified_for_learning_rate_targeted_senet[z][i][j] = accuracy_misclassified_senet
            else:
                print("ERRORE TOTAL IMAGE == 0 ERRORE")
        
    print("--------------------------------------------------------------------------------------------------------------------------------------------------------------------")
    print("")
    
                

ETICHETTA TARGET:   Aaron_Hernandez
Inizio Attacco CARLINI-WAGNER TARGETED
<---> Attacco con max_iter = 2| learning rate = 0.3 | initial cost = 300 <--->


C&W L_2: 100%|██████████| 1/1 [00:04<00:00,  4.66s/it]


Etichetta Corretta: Andrea_Pirlo | Predetto:  Andrea_Pirlo  probabilità 12.725250244140625 perturbazione applicata 0.0


C&W L_2: 100%|██████████| 1/1 [00:03<00:00,  3.27s/it]


Etichetta Corretta: Antonio_Cassano | Predetto:  Branch_Warren  probabilità 10.506441116333008 perturbazione applicata 0.0


...

Etichetta Corretta: Christopher_Nolan | Predetto:  Christopher_Nolan  probabilità 12.074438095092773 perturbazione applicata 0.0


C&W L_2: 100%|██████████| 1/1 [00:02<00:00,  2.44s/it]


Etichetta Corretta: Cristiano_Ronaldo | Predetto:  Cristiano_Ronaldo  probabilità 12.945646286010742 perturbazione applicata 0.0


C&W L_2: 100%|██████████| 1/1 [00:02<00:00,  2.33s/it]

Etichetta Corretta: Diego_Maradona | Predetto:  Diego_Maradona  probabilità 12.622477531433105 perturbazione applicata 0.0
--------------------------------------------------------------------------------------------------------------------------------------------------------------------



In [ ]:
print('Matrice Accuracy' + str(learning_rate))
for row_name, row_data in zip(initial_const, accuracy_for_learning_rate_targeted_resnet[0]):
    print(f'        {row_name }   ' + '  '.join(map(str, row_data)))


print('Matrice Perturbation' + str(learning_rate))
for row_name, row_data in zip(initial_const, perturbation_for_learning_rate_targeted):
    print(f'        {row_name}         ' + '  '.join(map(str, row_data)))


print('Matrice Misclassificazione' + str(learning_rate))
for row_name, row_data in zip(initial_const, accuracy_misclassified_for_learning_rate):
    print(f'        {row_name}         ' + '  '.join(map(str, row_data)))




In [16]:
print(accuracy_for_learning_rate_targeted_resnet[0])

print(accuracy_for_learning_rate_targeted_senet[0])

print(accuracy_misclassified_for_learning_rate_targeted_senet[0])

[[0. 0. 0.]
 [0. 0. 0.]]
[[0.  0.  0.9]
 [0.  0.  0.9]]
[[0. 0. 0.]
 [0. 0. 0.]]


In [ ]:
# plot accuracy/Attack strength

fig, ax = plt.subplots()
ax.plot(learning_rate, perturbation_for_learning_rate_targeted[0], 'b-', label='Perturbazione')
ax.plot(learning_rate, accuracy_for_learning_rate_targeted[0], 'r-', label='Accuracy')
#ax.plot(np.array(learning_rate), np.array(perturbation_for_learning_rate[1]), 'r-', label='max_iter == 2')
#ax.plot(np.array(learning_rate), np.array(perturbation_for_learning_rate[2]), 'g-.', label='max_iter == 4')

legend = ax.legend(loc='upper center', shadow=True, fontsize='large')
legend.get_frame().set_facecolor('#00FFCC')

plt.ylabel('Perturbation')
plt.xlabel('Learning Rate')



plt.tight_layout()

plt.show()

#### Caricamento di una sola immagine per i test

In [ ]:
filename = "test_set_cropped/Brad_Pitt_1_face_0.jpg"
test_img = load_image(filename)

print(test_img.shape)
print(test_img.size)
test_img = test_img.unsqueeze(0)
print(test_img.shape)
print(test_img.size)
test_img = test_img.numpy()
print(test_img.shape)
print(type(test_img))

In [ ]:
binary_search_steps = 5
confidence = 0.7
max_iter = 7
learning_rate = 0.001
initial_const = 350

target_class = 100
etichetta_target = LABELS[target_class]
print("ETICHETTA TARGET: ", etichetta_target)

targeted_labels = target_class*np.ones(LABELS.size)
one_hot_targeted_labels = tf.keras.utils.to_categorical(targeted_labels, num_classes = 8631)

attack = CarliniL2Method(classifier=classifier, binary_search_steps=binary_search_steps, confidence=confidence, max_iter=max_iter, learning_rate=learning_rate, initial_const=initial_const, targeted=True)
test_images_adv = attack.generate(test_img, one_hot_targeted_labels)
model_predictions = classifier.predict(test_images_adv)
correct_label = re.sub(r'_\d+_face_0\.jpg$', '', filename)   
perturbation.append(np.mean(np.abs((test_images_adv - test_img))))  
predicted_label = LABELS[np.array(model_predictions[0].argmax())]
print("Etichetta reale:{} || Predetto: {} con probabilità: {} e con perturbazione: {}".format(correct_label,predicted_label,model_predictions[0][model_predictions.argmax()],perturbation[-1]))
                    

In [ ]:
# Prepara le immagini per la visualizzazione
# Rimuovi la dimensione batch extra e converti nel formato channels-last

test_images_adv = np.squeeze(test_images_adv, axis=0)
test_images_adv = np.transpose(test_images_adv, (1, 2, 0))

# Converti le immagini in uint8 per la visualizzazione
if test_img.dtype != np.uint8:
    test_img_numpy = (test_img * 255).astype(np.uint8)
    test_img_numpy = np.squeeze(test_img_numpy, axis=0)  # Rimuovi la dimensione batch extra
    test_img_numpy = np.transpose(test_img_numpy, (1, 2, 0))

if test_images_adv.dtype != np.uint8:
    test_images_adv = (test_images_adv * 255).astype(np.uint8)

# Visualizza le immagini affiancate con Matplotlib


In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

# Mostra l'immagine originale
ax1.imshow(test_img_numpy)
ax1.set_title('Original Image')
ax1.axis('off')

# Mostra l'immagine avversaria
ax2.imshow(test_images_adv)
ax2.set_title(f'Adversarial Image\nPredicted: {predicted_label}')
ax2.axis('off')

# Mostra la figura
plt.suptitle("DeepFool Adversarial Images")
plt.show()